In [1]:
%load_ext autoreload
%autoreload 2

Using simpler models with the same results. Can this be considered a victory?

In [2]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('..')))

from utility.classification_utility import *

Let's start by building our beautiful dataset

In [3]:
cyc = '../dataset/cyclists_cleaned.csv'
races = '../dataset/races_cleaned.csv'
df = make_dataset_for_classification(races, cyc)

100.00%  


In [4]:
TO_USE_COLS = [
    # over time
    'total_points',
    'avg_points_per_race', 
    'average_position',
    'avg_speed_cyclist', 
    'mean_stamina_index',
    'race_count',
    # race related
    'length',
    'climb_total', 
    'profile', 
    'startlist_quality', 
    'cyclist_age_rac', 
    'steepness', 
    'is_tarmac', 
    'stage_type',
    # cyclist related
    'height',
    'weight',
    'bmi',
    'home_game',
    # for the split
    'date'
]

df = df[TO_USE_COLS]

df_tr, df_vl, df_ts = get_data_split(df)

df_tr = df_tr.drop(columns=['date'])
df_vl = df_vl.drop(columns=['date'])
df_ts = df_ts.drop(columns=['date'])

In [5]:
df_tr.describe()

,total_points,avg_points_per_race,average_position,avg_speed_cyclist,mean_stamina_index,race_count,length,climb_total,profile,startlist_quality,cyclist_age_rac,steepness,height,weight,bmi,home_game
count,355140.000000,355140.000000,355140.000000,355140.000000,355140.000000,355140.000000,355140.000000,249193.000000,266589.000000,355140.000000,355137.000000,249193.000000,321001.000000,320833.000000,320802.000000,355140.000000
mean,526.973419,2.887881,74.714507,10.874696,6.008749,141.057265,164550.249141,2363.854735,2.667893,1063.244140,28.695672,1.473980,179.613512,68.298944,21.146952,0.259022
std,1055.823747,4.023438,24.449370,1.103600,2.514212,123.319424,63364.769233,1387.534807,1.499612,317.350979,3.826340,0.904209,6.151154,6.080615,1.229202,0.438098
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,1900.000000,2.000000,1.000000,191.000000,13.000000,0.006865,159.000000,48.000000,17.505615,0.000000
25%,33.000000,0.576923,58.120958,10.841412,5.058801,43.000000,153000.000000,1302.000000,1.000000,866.000000,26.000000,0.869592,175.000000,64.000000,20.305175,0.000000
50%,156.000000,1.495050,74.361111,10.974416,6.431130,108.000000,176500.000000,2306.000000,2.000000,981.000000,28.000000,1.349307,180.000000,68.000000,21.107266,0.000000
75%,537.000000,3.563237,90.555667,11.093480,7.289545,207.000000,200400.000000,3347.000000,4.000000,1177.000000,31.000000,1.898734,184.000000,72.000000,21.877551,1.000000
max,14205.000000,99.500000,197.000000,15.837104,44.510839,836.000000,298000.000000,6939.000000,5.000000,1994.000000,56.000000,8.929032,204.000000,94.000000,26.775510,1.000000


In [6]:
df_vl.describe()

,total_points,avg_points_per_race,average_position,avg_speed_cyclist,mean_stamina_index,race_count,length,climb_total,profile,startlist_quality,cyclist_age_rac,steepness,height,weight,bmi,home_game
count,43480.000000,43480.000000,43480.000000,43480.000000,43480.000000,43480.000000,43480.000000,43480.000000,43480.000000,43480.000000,43480.000000,43480.000000,43271.000000,43160.000000,43159.000000,43480.000000
mean,688.369687,3.069957,77.133788,10.881195,7.300974,173.552001,162263.785189,2466.730727,2.931141,967.328473,28.859568,1.534132,180.576090,68.133612,20.870786,0.206302
std,1475.668263,5.086594,24.650337,1.090281,1.591573,154.321619,62168.745762,1360.960999,1.405714,305.254548,4.316586,0.754490,6.471154,6.598456,1.369468,0.404654
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,3870.000000,3.000000,1.000000,560.000000,19.000000,0.019108,159.000000,51.000000,16.412069,0.000000
25%,37.000000,0.681818,60.585882,10.833986,6.612263,45.000000,149800.000000,1492.000000,2.000000,765.000000,25.000000,1.015000,176.000000,63.000000,19.840664,0.000000
50%,193.000000,1.573121,78.582711,10.946337,7.010705,130.000000,175500.000000,2527.000000,3.000000,885.000000,28.000000,1.473986,181.000000,68.000000,20.811655,0.000000
75%,689.000000,3.613854,94.142248,11.074455,7.720542,266.000000,199600.000000,3448.000000,4.000000,1044.000000,32.000000,2.062842,185.000000,73.000000,21.777553,0.000000
max,15901.000000,225.000000,174.000000,15.961945,38.665821,781.000000,305000.000000,5848.000000,5.000000,1699.000000,42.000000,3.744538,204.000000,90.000000,25.308642,1.000000


In [7]:
df_ts.describe()

,total_points,avg_points_per_race,average_position,avg_speed_cyclist,mean_stamina_index,race_count,length,climb_total,profile,startlist_quality,cyclist_age_rac,steepness,height,weight,bmi,home_game
count,30429.000000,30429.000000,30429.000000,30429.000000,30429.000000,30429.000000,30429.000000,30429.000000,30429.000000,30429.000000,30429.000000,30429.000000,30063.000000,29784.000000,29721.000000,30429.000000
mean,724.557199,3.334084,73.407412,11.001439,7.400054,171.472937,160882.094712,2483.717473,2.872720,955.568077,28.780998,1.545741,180.913216,68.205698,20.821176,0.203326
std,1444.381982,5.187729,23.663753,1.241446,1.711949,156.646208,59556.292819,1338.604937,1.495359,350.047471,4.486702,0.806937,6.643433,6.679155,1.399868,0.402479
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,5120.000000,3.000000,1.000000,156.000000,19.000000,0.026087,158.000000,50.000000,16.604275,0.000000
25%,46.000000,0.750000,56.723014,10.968517,6.703523,49.000000,152600.000000,1649.000000,2.000000,749.000000,25.000000,1.040816,176.000000,63.000000,19.789698,0.000000
50%,206.000000,1.767296,74.836066,11.085279,7.154940,120.000000,172800.000000,2556.000000,2.000000,835.000000,28.000000,1.520656,181.000000,68.000000,20.745069,0.000000
75%,761.000000,4.002137,90.857143,11.218133,7.925009,261.000000,193000.000000,3471.000000,5.000000,1059.000000,32.000000,1.994347,185.000000,73.000000,21.718066,0.000000
max,16480.000000,150.000000,172.000000,14.793926,21.698706,827.000000,294000.000000,5852.000000,5.000000,1584.000000,42.000000,5.752688,202.000000,92.000000,25.308642,1.000000
